In [1]:
import gensim
import json
import re
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim 
import pickle

morph = MorphAnalyzer()

C:\Users\vasel\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Домашнее задание

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками**.

1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

7) заново обучите LDA c теми же параметрами (параметрами самой лучшей модели, заново перебирать не нужно);

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

Выполним нормализацию и сохраним тексты в формате vw:

In [3]:
texts = open('habr_texts.txt', encoding='utf-8').read().splitlines()

In [4]:
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def clean(words):
    clean = [morph.parse(word)[0].normal_form for word in words if len(set(word)) > 1]
    return clean

In [8]:
from collections import Counter
f = open('habr_texts.vw', 'w', encoding='utf-8')

for i, text in enumerate(texts):
    c = Counter(text)
    doc = 'doc_'+ str(i) + ' '
    vw_text = ' '.join([x+':'+str(c[x]) for x in c])
    
    f.write(doc + vw_text  + '\n')
f.close()

In [5]:
texts = [clean(word_tokenize(text.lower())) for text in texts]

Сделаем также нграммы, будем работать с ними:

In [6]:
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) 
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

Чтобы не лемматизировать тексты каждый раз заново, сохраним их и потом просто будем загружать:

In [5]:
"""
with open('lemmatized_texts.txt', 'wb') as f:
    pickle.dump(texts, f)"""
f=open('lemmatized_texts.txt', 'rb') 
lem_texts=f.read()
texts=pickle.loads(lem_texts)

In [6]:
"""with open('lemmatized_ngrams.txt', 'wb') as f1:
    pickle.dump(ngrammed_texts, f1)"""
f=open('lemmatized_ngrams.txt', 'rb') 
n_texts=f.read()
ngrammed_texts=pickle.loads(n_texts)

Создание словаря:

In [7]:
dictionary = gensim.corpora.Dictionary(ngrammed_texts)

In [8]:
dictionary.filter_extremes(no_above=0.25, no_below=40)
dictionary.compactify()

In [9]:
print(dictionary)

Dictionary(4894 unique tokens: ['.net', 'address', 'api', 'azure', 'class']...)


In [10]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [11]:
tfidf=gensim.models.TfidfModel(corpus, id2word=dictionary)
corpus = tfidf[corpus]

In [12]:
lda = gensim.models.LdaMulticore(corpus, 20, id2word=dictionary, passes=2)

Данные топики приведены для модели с использованием tfidf:

In [13]:
lda.print_topics()

[(0,
  '0.006*"gt" + 0.005*"lt" + 0.003*"int" + 0.003*"const" + 0.003*"if" + 0.002*"var" + 0.002*"new" + 0.002*"return" + 0.002*"файл" + 0.002*"пакет"'),
 (1,
  '0.002*"gt" + 0.002*"lt" + 0.002*"файл" + 0.002*"if" + 0.002*"the" + 0.002*"виртуальный" + 0.002*"номер" + 0.001*"google" + 0.001*"автомобиль" + 0.001*"сервер"'),
 (2,
  '0.002*"игра" + 0.001*"производство" + 0.001*"сеть" + 0.001*"объект" + 0.001*"программист" + 0.001*"модель" + 0.001*"орбита" + 0.001*"космический" + 0.001*"аккумулятор" + 0.001*"аппарат"'),
 (3,
  '0.002*"else" + 0.002*"gt" + 0.002*"игра" + 0.001*"файл" + 0.001*"оборудование" + 0.001*"книга" + 0.001*"смартфон" + 0.001*"lt" + 0.001*"продукт" + 0.001*"дата-центр"'),
 (4,
  '0.003*"gt" + 0.003*"lt" + 0.002*"php" + 0.002*"язык" + 0.002*"курс" + 0.002*"int" + 0.002*"amp" + 0.002*"android" + 0.002*"return" + 0.002*"let"'),
 (5,
  '0.003*"уязвимость" + 0.003*"доклад" + 0.002*"атака" + 0.002*"безопасность" + 0.002*"вредоносный" + 0.002*"microsoft" + 0.002*"gt" + 0.002*

Было обучено несколько моделей, менялись следующие параметры: количество тем, количество проходов (passes), фильтры (границы) словаря. Настоящая модель имеет параметры, представленные выше в ноутбуке (20 тем, 2 прохода, верхняя граница - 0.25, нижняя - 40).

С использованием tfidf-преобразования можно получить модель, обладающую следующими характеристиками:  
perplexity: -96.94593454206394   
coherence: 0.38519058302644316    
Ниже представлены три лучшие темы такой модели.


In [19]:
print(lda.print_topics()[0])
print(lda.print_topics()[1])
print(lda.print_topics()[2])

(0, '0.006*"gt" + 0.005*"lt" + 0.003*"int" + 0.003*"const" + 0.003*"if" + 0.002*"var" + 0.002*"new" + 0.002*"return" + 0.002*"файл" + 0.002*"пакет"')
(1, '0.002*"gt" + 0.002*"lt" + 0.002*"файл" + 0.002*"if" + 0.002*"the" + 0.002*"виртуальный" + 0.002*"номер" + 0.001*"google" + 0.001*"автомобиль" + 0.001*"сервер"')
(2, '0.002*"игра" + 0.001*"производство" + 0.001*"сеть" + 0.001*"объект" + 0.001*"программист" + 0.001*"модель" + 0.001*"орбита" + 0.001*"космический" + 0.001*"аккумулятор" + 0.001*"аппарат"')


Без использования tfidf-преобразования у модели с теми же параметрами характеристики будут следующими:    
perplexity: -14.204252506032987   
coherence: 0.37094050332241657    
Ниже представлены три лучшие темы такой модели.

In [68]:
print(lda.print_topics()[0])
print(lda.print_topics()[1])
print(lda.print_topics()[2])

(0, '0.004*"потому" + 0.004*"боль" + 0.004*"робот" + 0.004*"диск" + 0.004*"письмо" + 0.003*"сигнал" + 0.003*"энергия" + 0.003*"виртуальный" + 0.003*"очередь" + 0.003*"документ"')
(1, '0.016*"android" + 0.004*"print" + 0.004*"true" + 0.004*"сборка" + 0.004*"in" + 0.003*"шаблон" + 0.003*"адрес" + 0.003*"сигнал" + 0.003*"протокол" + 0.003*"слушать"')
(2, '0.004*"2016" + 0.004*"windows" + 0.004*"скидка" + 0.004*"услуга" + 0.004*"товар" + 0.004*"linux" + 0.004*"российский" + 0.003*"потому" + 0.003*"продажа" + 0.003*"виртуальный"')


У работавшей ранее визуализации тем в пятницу, кажется, выходной, поэтому лучше ориентироваться на текстовый список тем выше :(

In [28]:
pyLDAvis.enable_notebook()

In [29]:
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

C:\Users\vasel\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.041976 -0.005245       1        1  36.316528
17     0.028265 -0.000736       2        1  27.266226
8     -0.050599 -0.005312       3        1   7.395980
4     -0.013707  0.004905       4        1   7.185934
12    -0.025353 -0.002795       5        1   4.333282
5      0.004012  0.039346       6        1   4.243518
9      0.010614  0.015339       7        1   2.767937
19    -0.006432  0.016473       8        1   2.626609
7      0.005128 -0.002510       9        1   1.011944
14    -0.010560 -0.008595      10        1   0.981744
0     -0.029643  0.001400      11        1   0.835332
18    -0.003710 -0.007697      12        1   0.758534
11     0.006239 -0.007720      13        1   0.607533
1      0.003933  0.001109      14        1   0.598234
15     0.004262 -0.006329      15        1   0.551122
13    -0.008153 -0.010187      16        1   0.532088
16     0.010277 -0.008142      17        1   0.500338
10     0.010450 -0.001771      18        1   0.496140
3      0.004673  0.000721      19        1   0.495676
6      0.018326 -0.012254      20        1   0.495294, topic_info=     Category       Freq         Term      Total  loglift  logprob
term                                                              
11    Default  95.000000           gt  95.000000  30.0000  30.0000
270   Default  83.000000         игра  83.000000  29.0000  29.0000
15    Default  75.000000           lt  75.000000  28.0000  28.0000
751   Default  41.000000         файл  41.000000  27.0000  27.0000
1164  Default  44.000000         сайт  44.000000  26.0000  26.0000
194   Default  42.000000         язык  42.000000  25.0000  25.0000
1544  Default  48.000000         сеть  48.000000  24.0000  24.0000
693   Default  43.000000       сервер  43.000000  23.0000  23.0000
893   Default  42.000000       объект  42.000000  22.0000  22.0000
1457  Default  40.000000       модель  40.000000  21.0000  21.0000
359   Default  37.000000      продукт  37.000000  20.0000  20.0000
563   Default  46.000000       клиент  46.000000  19.0000  19.0000
3087  Default  32.000000       учёный  32.000000  18.0000  18.0000
1794  Default  40.000000           ты  40.000000  17.0000  17.0000
694   Default  39.000000       сервис  39.000000  16.0000  16.0000
423   Default  34.000000          amp  34.000000  15.0000  15.0000
14    Default  28.000000           if  28.000000  14.0000  14.0000
3241  Default  27.000000  космический  27.000000  13.0000  13.0000
695   Default  32.000000       сигнал  32.000000  12.0000  12.0000
1339  Default  32.000000        рынок  32.000000  11.0000  11.0000
205   Default  26.000000          the  26.000000  10.0000  10.0000
3707  Default  32.000000        игрок  32.000000   9.0000   9.0000
198   Default  34.000000       google  34.000000   8.0000   8.0000
1169  Default  28.000000     смартфон  28.000000   7.0000   7.0000
466   Default  29.000000       return  29.000000   6.0000   6.0000
1903  Default  26.000000          int  26.000000   5.0000   5.0000
1250  Default  34.000000        книга  34.000000   4.0000   4.0000
1041  Default  31.000000      элемент  31.000000   3.0000   3.0000
711   Default  35.000000    сотрудник  35.000000   2.0000   2.0000
691   Default  32.000000        связь  32.000000   1.0000   1.0000
...       ...        ...          ...        ...      ...      ...
4607  Topic20   0.153287      зеркало   7.540287   1.4121  -7.1442
2548  Topic20   0.216093           3d  11.050606   1.3732  -6.8008
2174  Topic20   0.279224        intel  15.021968   1.3225  -6.5445
1078  Topic20   0.338095         звук  20.286018   1.2134  -6.3532
1791  Topic20   0.354273  температура  21.923370   1.1825  -6.3064
2096  Topic20   0.172125       миссия   9.370543   1.3107  -7.0283
2859  Topic20   0.218642        wi-fi  13.082417   1.2162  -6.7890
1763  Topic20   0.249607      матрица  15.690390   1.1669  -6.6566
2349  Topic20   0.2552

Расчёт метрик подели с tfidf (представлены выше):

In [21]:
import numpy as np

In [22]:
lda.log_perplexity(corpus[:2000], total_docs=100)

-96.94593454206394

In [23]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                   texts=texts, 
                                                   dictionary=dictionary, coherence='c_v')

In [24]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [25]:
coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=texts, 
                                                   dictionary=dictionary, coherence='c_v')

In [26]:
coherence_model_lda.get_coherence()

0.38519058302644316

NMF:

In [92]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

При создании этой модели менялись такие параметры, как максимальное количество признаков и число компонентов. В итоге было применено максимальное количество признаков, равное 30000, и количество компонент, равное 30:

In [109]:
vectorizer = TfidfVectorizer(max_features=30000, min_df=15, max_df=0.3, lowercase=False)
X = vectorizer.fit_transform(stexts)

In [93]:
stexts = [' '.join(text) for text in texts]

In [110]:
model = NMF(n_components=30)

In [111]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=30, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [112]:
def get_nmf_topics(model, n_top_words):
    
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(30):

        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

Можно видеть, что первые три темы, особенно вторая и третья, имеют некоторое сходство с тремя лучшими темами предыдущей модели. В предыдущем примере также имелась тема, связанная с играми (третья в обоих случаях), хотя с использованием NMF она более ярко выражена; кроме того, имелась тема (в обоих случаях вторая), связанная с иноязычной терминологией.

In [113]:
get_nmf_topics(model, 15)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25,Topic # 26,Topic # 27,Topic # 28,Topic # 29,Topic # 30
0,ты,gt,игра,мозг,диск,рынок,камера,js,сайт,public,...,язык,int,файл,сервер,бот,робот,запрос,studio,vr,сигнал
1,продукт,lt,игрок,учёный,процессор,российский,смартфон,react,страница,string,...,обучение,if,docker,центр,telegram,автомобиль,end,pvs,виртуальный,звук
2,сотрудник,div,игровой,пациент,память,товар,устройство,css,реклама,void,...,алгоритм,amp,php,дата,сообщение,ребёнок,select,анализатор,реальность,частота
3,бизнес,class,играть,клетка,intel,рубль,экран,javascript,клиент,new,...,программирование,lt,сервер,инфраструктура,чат,машина,from,visual,oculus,усилитель
4,программист,name,персонаж,заболевание,ssd,россия,телефон,angular,браузер,this,...,модель,void,http,услуга,телеграм,робототехника,as,net,шлем,устройство
5,клиент,this,steam,исследование,производительность,страна,аккумулятор,веб,контент,return,...,программа,std,sudo,облачный,bot,беспилотный,null,ошибка,vive,наушник
6,надо,props,геймплей,болезнь,ядро,магазин,видео,vue,домен,класс,...,нейросеть,char,etc,клиент,api,lego,таблица,ide,машина,ток
7,да,html,unity,ген,накопитель,цена,дисплей,браузер,письмо,private,...,объект,return,nginx,облако,мессенджер,датчик,if,инструмент,htc,напряжение
8,опыт,return,жанр,врач,тест,скидка,регистратор,компонент,рекламный,var,...,изображение,for,скрипт,сервис,канал,дрон,varchar,плагин,unreal,искажение
9,менеджер,type,движок,организм,сервер,налог,карта,dom,сервис,class,...,сеть,const,usr,оборудование,message,движение,set,core,дополнить,диапазон
